In [ ]:
using MakieCore
import MakieCore: convert_arguments 

# This part would be the extension package (calls like lines, scatter, etc.)
using CairoMakie

import NamedTrajectories as NT

- convert_arguments is not good because it cannot plot multiple lines

In [ ]:
function convert_arguments(
    P::MakieCore.PointBased, 
    traj::NT.NamedTrajectory{<:Any},
    component::Int,
)
    times = NT.get_times(traj)
    positions = [(times[j], traj.data[component, j]) for j in eachindex(times)]
    return convert_arguments(P, positions)
end

# define the default plot type (preserve `plot` calls)
MakieCore.plottype(::NT.NamedTrajectory, ::Int) = Lines


# NOTE: The rest of these conversions should be recipes!
# -----

# function convert_arguments(
#     P::MakieCore.PointBased, 
#     traj::NT.NamedTrajectory{<:Any},
#     name::Symbol,
#     index::Int
# )
#     times = NT.get_times(traj)
#     positions = [(times[j], traj[name][index, j]) for j in eachindex(times)]
#     return convert_arguments(P, positions)
# end

# function convert_arguments(
#     P::MakieCore.PointBased, 
#     traj::NT.NamedTrajectory{<:Any},
#     name::Symbol
# )
#     times = NT.get_times(traj)
#     rows, cols = size(traj[name])
#     positions = vec([Point2(times[j], traj[name][i, j]) for i in 1:rows, j in 1:cols])
#     return convert_arguments(P, positions)
#     # return vec(positions)
# end


In [ ]:
function convert_arguments(
    P::MakieCore.PointBased, 
    traj::NT.NamedTrajectory{<:Any},
    comps::AbstractVector{Int},
    transformation::Function
)
    times = NT.get_times(traj)
    positions = [(times[j], transformation(traj.data[comps, j])) for j in eachindex(times)]
    return convert_arguments(P, positions)
end

In [ ]:
traj = rand(NT.NamedTrajectory, 10);

In [ ]:
f = Figure()
ax = Axis(f[1, 1])
[lines!(ax, traj, i) for i in traj.components[:u]]
f

Notes
- Makie defaults to Makie.Axis as the preferred axis.

In [ ]:
@recipe(PlotComponents, traj, components) do scene
    Theme(
        Lines = (
            linewidth = 10,
            linestyle = :dash,
        ),
        Axis = (
            leftspinevisible = false,
        ),
    )
    # Attributes(
    #     labels = false,
    # )
end

# Default is to plot on axis
Makie.preferred_axis_type(P::PlotComponents) = Makie.Axis

# Dealing with a default plot of an existing component
function Makie.plot!(P::PlotComponents{<:Tuple{<:NT.NamedTrajectory, Symbol}})
    lift(P[:traj], P[:components]) do traj, name
        for i in traj.components[name]
            lines!(P, traj, i, label="$(name)$(i)")
        end
    end
    return P
end

# Ability to add a legend
Makie.get_plots(P::PlotComponents) = P.plots

# Dealing with a transformation to a new component
function Makie.plot!(
    P::PlotComponents{<:Tuple{<:NT.NamedTrajectory, <:Tuple{Symbol, Symbol, <:Function}}}
)
    lift(P[:traj], P[:components]) do traj, transformation_data
        input_name, output_name, transformation = transformation_data
        times = NT.get_times(traj)

        # Descriptive error message
        output_data = try
            stack(transformation.(eachcol(traj[input_name])))
        catch
            throw(ArgumentError("Transformation $(input_name) -> $(output_name) failed."))
        end

        for (j, output_row) in enumerate(eachrow(output_data))
            lines!(P, times, output_row, label="$(output_name)$(j)")
        end
    end
    return P
end

# How to deal with multiple symbols?

In [ ]:
f = Figure()
ax = Axis(f[1, 1])

plotcomponents!(ax, traj, (:x, :y, x -> abs.(x)))
Legend(f[1, 2], ax)

plotcomponents(f[2, 1], traj, :u)
f